In [ ]:
import pandas as pd
import numpy as np
import json
from itertools import combinations
from collections import defaultdict

from matplotlib import pyplot as plt

## Getting data and computing basic statistics

In [ ]:
# Main data file
df = pd.read_csv('../data/evaluations.csv', sep='\t')

# Each row is one participant:
# + Columns with names {Paper1, Paper2, ..., Paper10} -- papers the participant chose to include in the dataset
# (NaNs stand for cases when a participant reported less than 10)
# + Columns with names {Expertise1, Expertise2, ..., Expertise10} -- values of expertise reported by the participant
df.head()

In [ ]:
# Getting participants
participants = set(df['ParticipantID'])

# Getting papers
papers = set()

for x in range(1, 11):
    tmp_papers = set(df[~pd.isna(df[f'Paper{x}'])][f'Paper{x}'])
    papers = papers.union(tmp_papers)

# Translating df from csv to dict of the form {participant: {Paper1: Expertise1, Paper2: Expertise2, ...}}
data = {}

for idx, row in df.iterrows():

    key = str(row['ParticipantID'])

    data[key] = {row[f'Paper{x}']: row[f'Expertise{x}'] for x in range(1, 11)
                                                            if not pd.isna(row[f'Paper{x}'])}

In [ ]:
print(f"There are {len(participants)} researchers who contributed to the dataset")
print(f"{len(papers)} unique papers are included in the dataset")
print(f"The dataset consists of {sum([len(data[key]) for key in data])} datapoints")

## More data on participants

In [ ]:
# Getting Semantic Scholar profiles of participants
rev_profiles = {}

for rev in participants:
    with open(f'../data/participants/{rev}.json', 'r') as handler:
        rev_profiles[rev] = json.load(handler)

# Getting bibliography lengths
lens = [len(rev_profiles[rev]['papers']) for rev in rev_profiles]

print(f"Minimum number of publications: {min(lens)}")
print(f"Maximum number of publications: {max(lens)}")
print(f"Mean number of publications: {round(np.mean(lens))}")
print(f"Median number of publications: {np.median(lens)}")

# Other demographic statistics are computed by manual inspections of participants' websites

## More data on papers

In [ ]:
# Getting Semantic Scholar profiles of papers
pap_profiles = {}

for pap in papers:
    with open(f'../data/papers/{pap}.json', 'r') as handler:
        pap_profiles[pap] = json.load(handler)

# Research areas as classified by Semantic Scholar
with open('../data/research_areas.json', 'r') as handler:
    research_areas = json.load(handler)

# Count how many times each paper appears in the participants' reports
pap_counter = defaultdict(lambda: 0)

for rev in data:
    for pap in data[rev]:
        pap_counter[pap] += 1

print('Dataset statistics:')
for x in [1, 2, 3]:
    print(f'|--> {len([pap for pap in pap_counter if pap_counter[pap] == x])} papers appear {x} time(s)')

In [ ]:
# Getting statistics for Table 2
ss_counter, arxiv_counter, pdf_counter, cs_counter, old_counter = 0, 0, 0, 0, 0

for key, paper in pap_profiles.items():

    if paper['ssId'] is not None:
        ss_counter += 1

    if paper['arXivId'] is not None:
        arxiv_counter += 1

    if paper['openAccess']:
        pdf_counter += 1

    if paper['ssId'] is not None and 'Computer Science' in research_areas[paper['ssId']]:
        cs_counter += 1

    if paper['year'] < 2020:
        old_counter += 1

# adding 1 to the counter of CS papers as the paper without a Semantic Scholar identifier belongs to the CS area
cs_counter += 1

In [ ]:
print(f"{ss_counter} papers are available on Semantic Scholar")
print(f"{arxiv_counter} papers are available on arXiv")
print(f"{pdf_counter} papers have their PDFs publicly available")
print(f"{cs_counter} papers belong to the CS area")
print(f"{round(100 - 100 * old_counter / len(papers))}% of papers are published on or after 2020")

## More data on self-reported evaluations

In [ ]:
stat1, stat2, stat3 = 0, 0, 0

responses_per_reviewer = [len(data[rev]) for rev in data]

stat1 = sum(responses_per_reviewer)
stat2 = np.mean(responses_per_reviewer)
stat3 = len([x for x in responses_per_reviewer if x >= 5])

print(f"There are {stat1} evaluations in total")
print(f"Mean number of evaluations per participant: {round(stat2, 1)}")
print(f"Number of participants with >= 5 evaluations: {stat3}")

### Marginal Expertise: Figure 1(a)

In [ ]:
# Bin values of expertise into 4 groups
marginal_expertise = {'[1, 2]': 0, '(2, 3]': 0, '(3, 4]': 0, '(4, 5]': 0}

for rev in data:
    for pap in data[rev]:

        # Expertise of reviewer `rev` in reviewing paper `pap`
        v = data[rev][pap]

        if v <= 2:
            marginal_expertise['[1, 2]'] += 1
        elif v <= 3:
            marginal_expertise['(2, 3]'] += 1
        elif v <= 4:
            marginal_expertise['(3, 4]'] += 1
        elif v <= 5:
            marginal_expertise['(4, 5]'] += 1
        else:
            raise ValueError("Wrong value of expertise")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

ax.bar([1.5, 2.5, 3.5, 4.5], [marginal_expertise['[1, 2]'],
                              marginal_expertise['(2, 3]'],
                              marginal_expertise['(3, 4]'],
                              marginal_expertise['(4, 5]']], width=0.8)

_ = ax.xaxis.set_ticks([1.5, 2.5, 3.5, 4.5])
_ = ax.set_xticklabels(["[1, 2]", "(2, 3]", "(3, 4]", "(4, 5]"])
_ = ax.tick_params(axis='both', labelsize=20)
ax.set_xlabel("Value of expertise", fontsize=25)
ax.set_ylabel("Count", fontsize=25)

plt.savefig('../figures/marginal_expertise.pdf', bbox_inches='tight')

### Pairwise differences in expertise: Figure 1(b)

In [ ]:
# Compute histogram of pairwise differences in expertise values reported by participants
diffs = defaultdict(lambda: 0)

for rev in data:
        for p1, p2 in combinations(data[rev].keys(), 2):
            diffs[np.abs(data[rev][p1] - data[rev][p2])] += 1

diffs_vals = sorted(list(diffs.keys()))

frequencies = [diffs[x] for x in diffs_vals]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

ax.bar(diffs_vals, frequencies, width=0.2)
_ = ax.tick_params(axis='both', labelsize=20)
ax.set_xlabel("Difference in expertise", fontsize=25)
ax.set_ylabel("Count", fontsize=25)

plt.savefig('../figures/difference_expertise.pdf', bbox_inches='tight')